<a href="https://colab.research.google.com/github/IamToef/FaceRecogNCKH/blob/main/FaceRegconizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install insightface
!pip install onnxruntime

In [ ]:
import torch
import torch .nn as nn

from PIL import Image
from torch.utils.data import Subset , DataLoader, Dataset
from torchvision.datasets import LFWPeople
from torchvision.models import resnet34
from torchvision import transforms
from sklearn.model_selection import train_test_split
from insightface.model_zoo import get_model

In [ ]:
lfw_dataset = LFWPeople(root='./data', split='train',  download=True)
lfw_dataset

Files already downloaded and verified


Dataset LFWPeople
    Number of datapoints: 9525
    Root location: ./data/lfw-py
    Alignment: funneled
    Split: train
    Classes (identities): 5749

In [ ]:
num_samples = len(lfw_dataset)
indices = list(range(num_samples))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)

In [ ]:
train_subset = Subset(lfw_dataset, train_idx)
test_subset = Subset(lfw_dataset, test_idx)

print(f'Training set size: {len(train_subset)}')
print(f'Test set size: {len(test_subset)}')

Training set size: 7620
Test set size: 1905


**(a) Xây dựng hàm tiền xử lý dữ liệu hình ảnh**

In [ ]:
IMG_SIZE = 64
img_transforms = transforms.Compose ([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5]
    )
])

:**(b) Xây dựng class LFWDataset**

In [ ]:
class LFWDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        original_idx = self.data.indices[idx]
        image, label = self.data.dataset[original_idx]

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.long)

        return image, label

**(c) Khai báo dataloader**

In [ ]:
TRAIN_BATCH_SIZE = 512
VAL_BATCH_SIZE = 256

train_dataset = LFWDataset(data=train_subset, transform=img_transforms)
test_dataset = LFWDataset(data=test_subset, transform=img_transforms)
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=VAL_BATCH_SIZE, shuffle=False)

**5. Xây dựng mô hình**

In [ ]:
class LFWModel(nn.Module):
    def __init__(self, n_classes):
        super(LFWModel, self).__init__()
        resnet = resnet34(pretrained=True)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        for param in self.backbone.parameters():
            param.requires_grad = False
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        in_features = resnet.fc.in_features
        self.fc = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.backbone(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
N_CLASSES = 5749
model = LFWModel(N_CLASSES).to(device)
test_input = torch.rand(1, 3, 224, 224).to(device)
with torch.no_grad():
  output = model(test_input)
  print(output.shape)

torch.Size([1, 5749])


In [ ]:
EPOCH = 10
LR = 1e-3
WEIGHT_DECAY= 1e-5

optimizer = torch.optim.Adam(model.parameters(), lr = LR, weight_decay = WEIGHT_DECAY)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(EPOCH):
  train_losses = []
  model.train()
  for images, labels in train_loader:
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)

    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

  train_loss = sum(train_losses)/len(train_losses)

  val_losses = []
  model.eval()
  with torch.no_grad():
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)

      outputs = model(images)
      loss = criterion(outputs, labels)

      val_losses.append(loss.item())

  val_loss = sum(val_losses)/len(val_losses)

  print(f'EPOCH {epoch + 1}:\tTrain loss: {train_loss: .3f}\tVal loss: {val_loss: .3f}')

EPOCH 1:	Train loss:  8.742	Val loss:  8.420
EPOCH 2:	Train loss:  6.436	Val loss:  8.646
EPOCH 3:	Train loss:  4.804	Val loss:  8.571
EPOCH 4:	Train loss:  3.469	Val loss:  8.591
EPOCH 5:	Train loss:  2.543	Val loss:  8.645
EPOCH 6:	Train loss:  1.967	Val loss:  8.824
EPOCH 7:	Train loss:  1.610	Val loss:  9.000
EPOCH 8:	Train loss:  1.362	Val loss:  9.175
EPOCH 9:	Train loss:  1.181	Val loss:  9.304
EPOCH 10:	Train loss:  1.045	Val loss:  9.429


In [ ]:
SAVE_PATH = 'FR_weights.pt'
torch.save(model.state_dict(), SAVE_PATH)